Group 2 - Term Project - 3250 Foundations of Data Science 

The below are the imports for this notebook. Run before doing anything else:

In [1]:
import pandas as pd
import xlrd
import os

In [2]:
# This is used to pull data from the Government of Canada weather site
# From what I could find, the site only allowed you to pull data for one month
# at a time. Given this I used a loop to update the URL with a new month/year
# after each download.
weather_year = '2010'
weather_canada = pd.DataFrame()

while int(weather_year) <= 2019:
    weather_month = '1'
    while int(weather_month) <= 12:
        weather_canada_url =  ("https://climate.weather.gc.ca/climate_data/"
                               "bulk_data_e.html?format=csv&stationID=31688&"
                               "Year=" + weather_year + 
                               "&Month=" + weather_month +
                               "&Day=1&timeframe=1&submit=Download+Data"
                               )
        weather_canada_new = pd.read_csv(weather_canada_url)
        weather_canada = pd.concat([weather_canada,weather_canada_new])
        weather_month = str(int(weather_month)+1)
    weather_year = str(int(weather_year) + 1)

weather_canada.head(5)

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Time,Temp (°C),...,Wind Spd Flag,Visibility (km),Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather
0,-79.4,43.67,TORONTO CITY,6158355,2010-01-01 00:00,2010,1,1,00:00,1.5,...,M,NaN,NaN,NaN,M,NaN,NaN,NaN,NaN,NaN
1,-79.4,43.67,TORONTO CITY,6158355,2010-01-01 01:00,2010,1,1,01:00,1.8,...,M,NaN,NaN,NaN,M,NaN,NaN,NaN,NaN,NaN
2,-79.4,43.67,TORONTO CITY,6158355,2010-01-01 02:00,2010,1,1,02:00,1.8,...,M,NaN,NaN,NaN,M,NaN,NaN,NaN,NaN,NaN
3,-79.4,43.67,TORONTO CITY,6158355,2010-01-01 03:00,2010,1,1,03:00,1.7,...,M,NaN,NaN,NaN,M,NaN,NaN,NaN,NaN,NaN
4,-79.4,43.67,TORONTO CITY,6158355,2010-01-01 04:00,2010,1,1,04:00,1.9,...,M,NaN,NaN,NaN,M,NaN,NaN,NaN,NaN,NaN


In [3]:
# create a date column by slicing the date and time column, 
# then delete the date and time column
weather_canada['date'] = weather_canada['Date/Time'].str[:10]
#weather_canada.drop(columns=['Date/Time'],inline=True)

weather_canada.head(5)

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Time,Temp (°C),...,Visibility (km),Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather,date
0,-79.4,43.67,TORONTO CITY,6158355,2010-01-01 00:00,2010,1,1,00:00,1.5,...,NaN,NaN,NaN,M,NaN,NaN,NaN,NaN,NaN,2010-01-01
1,-79.4,43.67,TORONTO CITY,6158355,2010-01-01 01:00,2010,1,1,01:00,1.8,...,NaN,NaN,NaN,M,NaN,NaN,NaN,NaN,NaN,2010-01-01
2,-79.4,43.67,TORONTO CITY,6158355,2010-01-01 02:00,2010,1,1,02:00,1.8,...,NaN,NaN,NaN,M,NaN,NaN,NaN,NaN,NaN,2010-01-01
3,-79.4,43.67,TORONTO CITY,6158355,2010-01-01 03:00,2010,1,1,03:00,1.7,...,NaN,NaN,NaN,M,NaN,NaN,NaN,NaN,NaN,2010-01-01
4,-79.4,43.67,TORONTO CITY,6158355,2010-01-01 04:00,2010,1,1,04:00,1.9,...,NaN,NaN,NaN,M,NaN,NaN,NaN,NaN,NaN,2010-01-01


In [4]:
# delete all empty columns
weather_canada = weather_canada.dropna(how='all', axis=1)


In [5]:
# Read in and clean BNE data

bne_url = ("https://opendata.arcgis.com/datasets/"
           "8ab59b498f6d4eae8ec631a287550376_0.csv?"
           "outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D"
           )

bne_data = pd.read_csv(bne_url)

bne_data.head(5)

,X,Y,Index_,event_unique_id,occurrencedate,reporteddate,premisetype,ucr_code,ucr_ext,offence,...,occurrencedayofyear,occurrencedayofweek,occurrencehour,MCI,Division,Hood_ID,Neighbourhood,Long,Lat,ObjectId
0,-79.582176,43.661335,714,GO-20141857431,2014-04-09T13:30:00.000Z,2014-04-09T13:39:00.000Z,House,2120,200,B&E,...,99.0,Wednesday,13,Break and Enter,D22,11,Eringate-Centennial-West Deane (11),-79.582176,43.661335,1
1,-79.362968,43.773071,715,GO-20141859201,2014-04-09T15:00:00.000Z,2014-04-09T18:19:00.000Z,House,2120,200,B&E,...,99.0,Wednesday,15,Break and Enter,D33,47,Don Valley Village (47),-79.362968,43.773071,2
2,-79.332382,43.681484,716,GO-20141866077,2014-04-10T13:00:00.000Z,2014-04-10T19:11:00.000Z,House,2120,220,B&E W'Intent,...,100.0,Thursday,13,Break and Enter,D55,69,Blake-Jones (69),-79.332382,43.681484,3
3,-79.386787,43.670227,719,GO-20141915866,2014-04-18T10:16:00.000Z,2014-04-18T18:00:00.000Z,Commercial,2120,200,B&E,...,108.0,Friday,10,Break and Enter,D53,98,Rosedale-Moore Park (98),-79.386787,43.670227,4
4,-79.381180,43.725376,724,GO-20141965079,2014-04-26T20:51:00.000Z,2014-04-26T20:51:00.000Z,House,2120,200,B&E,...,116.0,Saturday,20,Break and Enter,D53,41,Bridle Path-Sunnybrook-York Mills (41),-79.381180,43.725376,5


In [6]:
# Need to create a column to merge the data on. This will be a date and time
# column. Currently the date and time column (occurancedate) is formatted 
# differently than the date and time column in the weather data. In addition
# to this the occurance time for the bne data is to the minute where the
# weather data is hourly. Given this we need to round the bne time to the
# nearest hour.

def round_time(time_str):
    '''(str) -> (str)
    
    Return the time rounded to the nearest hour with the time_str provided.
    
    >>> round_time('13:30')
    '14:00'
    
    >>> round_time('13:20')
    '13:00'
    
    >>> round_time('24:40')
    '00:00'
    '''
    if int(time_str[3:]) < 30:
        return str(time_str[:2]) + ':00'
    else:
        if int(time_str[:2]) == 24:
            return '00:00'
        else:
            return str(int(time_str[:2]) + 1) + ':00'

In [7]:
# This creates the Dats/Time column for the and uses the round function
# created above.
bne_data['Date/Time'] = (bne_data['occurrencedate'].str[:10] + ' ' +
                         bne_data['occurrencedate'].str[11:16]
                         .apply(round_time)
                         )

In [8]:
# Merge the weather and BNE data set using the column 'Date/Time'
# filling everything from the bne_data (how='left'). There are stil
# some empty bne rows with no weather data - not sure why this is as 
# the weather data exists for those dates and times.
bne_weather_data = pd.merge(bne_data, weather_canada,how='left', 
                            on='Date/Time')

bne_weather_data.head(5)

,X,Y,Index_,event_unique_id,occurrencedate,reporteddate,premisetype,ucr_code,ucr_ext,offence,...,Temp (°C),Temp Flag,Dew Point Temp (°C),Rel Hum (%),Wind Dir Flag,Wind Spd Flag,Stn Press (kPa),Stn Press Flag,Hmdx,date
0,-79.582176,43.661335,714,GO-20141857431,2014-04-09T13:30:00.000Z,2014-04-09T13:39:00.000Z,House,2120,200,B&E,...,8.8,NaN,-4.8,38.0,M,M,100.03,NaN,NaN,2014-04-09
1,-79.362968,43.773071,715,GO-20141859201,2014-04-09T15:00:00.000Z,2014-04-09T18:19:00.000Z,House,2120,200,B&E,...,8.6,NaN,-5.5,36.0,M,M,100.01,NaN,NaN,2014-04-09
2,-79.332382,43.681484,716,GO-20141866077,2014-04-10T13:00:00.000Z,2014-04-10T19:11:00.000Z,House,2120,220,B&E W'Intent,...,14.8,NaN,2.1,42.0,M,M,99.34,NaN,NaN,2014-04-10
3,-79.386787,43.670227,719,GO-20141915866,2014-04-18T10:16:00.000Z,2014-04-18T18:00:00.000Z,Commercial,2120,200,B&E,...,8.9,NaN,-0.5,52.0,M,M,101.22,NaN,NaN,2014-04-18
4,-79.381180,43.725376,724,GO-20141965079,2014-04-26T20:51:00.000Z,2014-04-26T20:51:00.000Z,House,2120,200,B&E,...,5.7,NaN,-0.6,64.0,M,M,100.40,NaN,NaN,2014-04-26


In [9]:
# BnE broken down by premises type. Does anyone know how to get this as a 
# percentage of total BnE? It also makes sense to 'normalize' this - how 
# many apartments/commercial/houses are in the area?
crime_by_premise = bne_weather_data.pivot_table('ObjectId', aggfunc = 'count', 
                                                columns='premisetype')

In [10]:
bypremise3 = bne_weather_data.groupby('premisetype')['ObjectId'].count()

In [11]:
# BnE broken down by premises type. Does anyone know how to get this as a 
# percentage of total BnE? 
bypremise3/bypremise3.sum()*100

premisetype
Apartment     26.899450
Commercial    31.596693
House         35.384047
Other          6.064385
Outside        0.055425
Name: ObjectId, dtype: float64